### 1. Data Collection & Web Scraping

In [71]:
!pip install --upgrade python-youtube --quiet
!pip install git+https://github.com/openai/whisper.git --quiet
!pip install pytube --quiet

from pyyoutube import Client
from pytube import YouTube
import whisper
import csv
import pandas as pd
import isodate
import json
import os

model = whisper.load_model("base")

f = open('secrets.json')
secrets = json.load(f)
f.close()

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [86]:
channelId = "UCBJycsmduvYEL83R_U4JriQ"
playlistId = "UU" + channelId[2:]
maxVideos = 20

In [119]:
if (not os.path.exists('/content/video_details.csv')):
  videos_list = []
  comments_list = []
  client = Client(api_key=secrets['apiKey'])
  playlist_response = client.playlistItems.list(
      part="snippet",
      playlist_id=playlistId,
      maxResults=50,
      return_json = True
      )
  while True:
    videos_id = []

    for item in playlist_response['items']:
      videos_id += [item['snippet']['resourceId']['videoId']]

    video_response = client.videos.list(
        part="snippet, contentDetails, statistics",
        video_id=videos_id,
        return_json = True
        )

    for item in video_response['items']:
      if (isodate.parse_duration(item.get('contentDetails', 'PT0S').get('duration', 'PT0S')).days > 0 or isodate.parse_duration(item.get('contentDetails', 'PT0S').get('duration', 'PT0S0')).seconds > 60): # Youtube Shorts have a maximum duration of 60 seconds
        #yt = YouTube(f"https://www.youtube.com/watch?v={item['id'])}")
        #audio_stream = yt.streams.get_audio_only()
        #default_filename = audio_stream.download("/content")

        #transcript = model.transcribe(default_filename)

        videos_list += [(
            item.get('snippet', '').get('channelTitle', ''),
            item.get('snippet', '').get('channelId', ''),
            item.get('snippet', '').get('title', ''),
            item.get('id', ''),
            item.get('snippet', '').get('publishedAt', ''),
            item.get('contentDetails', '').get('duration', ''),
            item.get('statistics', '').get('viewCount', ''),
            item.get('statistics', '').get('likeCount', ''),
            item.get('statistics', '').get('commentCount', ''),
            item.get('snippet', '').get('tags', ''),
            #transcript.text
            )]

        comments_response = client.commentThreads.list(
          part="snippet,replies",
          maxResults=100,
          order="relevance",
          return_json = True,
          video_id=item['id']
        )
        while True:
          for comment in comments_response['items']:
            comments_list += [(
                item.get('snippet', '').get('channelTitle', ''),
                item.get('snippet', '').get('channelId', ''),
                item.get('snippet', '').get('title', ''),
                item.get('id', ''),
                comment.get('snippet', '').get('topLevelComment', '').get('snippet', '').get('authorDisplayName', ''),
                comment.get('snippet', '').get('topLevelComment', '').get('snippet', '').get('authorChannelId', '').get('value', ''),
                comment.get('snippet', '').get('topLevelComment', '').get('id', ''),
                comment.get('snippet', '').get('topLevelComment', '').get('id', ''),
                comment.get('snippet', '').get('topLevelComment', '').get('snippet', '').get('textOriginal', ''),
                comment.get('snippet', '').get('topLevelComment', '').get('snippet', '').get('publishedAt', ''),
                comment.get('snippet', '').get('topLevelComment', '').get('snippet', '').get('updatedAt', ''),
                comment.get('snippet', {}).get('topLevelComment', {}).get('snippet', {}).get('likeCount', 0)
                )]
            for replies in comment.get('replies', {}).get('comments', []):
              comments_list += [(
                  item.get('snippet', '').get('channelTitle', ''),
                  item.get('snippet', '').get('channelId', ''),
                  item.get('snippet', '').get('title', ''),
                  item.get('id', ''),
                  replies.get('snippet', '').get('authorDisplayName', ''),
                  replies.get('snippet', '').get('authorChannelId', '').get('value', ''),
                  comment.get('snippet', '').get('topLevelComment', '').get('id', ''),
                  replies.get('id', ''),
                  replies.get('snippet', '').get('textOriginal', ''),
                  replies.get('snippet', '').get('publishedAt', ''),
                  replies.get('snippet', '').get('updatedAt', ''),
                  replies.get('snippet', 0).get('likeCount', 0)
                  )]
          if not comments_response.get('nextPageToken', ''):
            break
          comments_response = client.commentThreads.list(
            part="snippet,replies",
            maxResults=100,
            order="relevance",
            return_json = True,
            video_id=item['id'],
            page_token=comments_response['nextPageToken']
          )

        if (len(videos_list)>=maxVideos):
          break

    if (len(videos_list)>=maxVideos):
      break
    else:
      playlist_response = client.playlistItems.list(
          part="snippet",
          playlist_id=playlistId,
          maxResults=50,
          return_json = True,
          page_token=playlist_response['nextPageToken']
          )

  with open('video_details.csv', 'w', newline='') as out:
      csv_out=csv.writer(out)
      csv_out.writerow([
          'Channel Name',
          'Channel ID',
          'Video Title',
          'Video ID',
          'Date Published',
          'Video Length',
          'No. Views',
          'No. Likes',
          'No. Comments',
          'Tags'
          #'Transcript'
          ])
      for row in videos_list:
          csv_out.writerow(row)

  with open('video_comments.csv', 'w', newline='') as out:
      csv_out=csv.writer(out)
      csv_out.writerow([
          'Channel Name',
          'Channel ID',
          'Video Title',
          'Video ID',
          'Author Name',
          'Authoer ID',
          'Parent Comment ID',
          'Comment ID',
          'Text',
          'Date Published',
          'Date Updated',
          'No. Likes',
          #'Transcript'
          ])
      for row in comments_list:
          csv_out.writerow(row)
video_df = pd.read_csv('/content/video_details.csv')
comment_df = pd.read_csv('/content/video_comments.csv')

In [122]:
video_df

,Channel Name,Channel ID,Video Title,Video ID,Date Published,Video Length,No. Views,No. Likes,No. Comments,Tags
0,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,Talking Tech and AI with Tim Cook!,pMX2cQdPubk,2024-06-12T15:56:11Z,PT16M33S,1915830,96902,7070,"['Tim Cook', 'Tim Cook Interview', 'Apple Inte..."
1,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,WWDC 2024 Recap: Is Apple Intelligence Legit?,qkYoBNdcXBU,2024-06-11T07:28:19Z,PT18M23S,4864158,195379,11776,"['WWDC', 'WWDC 2024', 'Apple Intelligence', 'A..."
2,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,I Visited Apple's Secret iPhone Testing Labs!,8reaJG7z-is,2024-06-07T21:31:09Z,PT10M52S,2639051,118160,6190,NaN
3,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,Sonos Ace Headphones: You Had One Job!,6_3cLlSlTKw,2024-06-03T14:57:33Z,PT11M23S,2315103,77770,4183,"['Sonos Ace', 'Sonos headphones', 'Sonos', 'MK..."
4,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,Android 15 Hands-On: Top 5 Features!,kkX8_nbBqBQ,2024-05-28T21:40:15Z,PT11M26S,2845227,113266,6546,"['Android 15', 'Android 15 features', 'Android..."
5,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,I Gave A Commencement Speech!,a7ItChBrY8E,2024-05-23T19:41:23Z,PT8M58S,1208357,82351,3832,"['MKBHD', 'commencement', 'commencement speech..."
6,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,5 Weird iPad Pro (M4) Decisions,PHcDrXeyguM,2024-05-22T02:26:31Z,PT13M2S,3764232,121059,6009,"['iPad Pro', 'iPad Pro M4', 'iPad Pro review',..."
7,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,M4 iPad Pro Impressions: Well This is Awkward,-T0MGehwWvE,2024-05-08T02:06:33Z,PT12M51S,6078343,183538,12941,"['M4 iPad', 'M4 iPad Pro', 'iPad Pro', 'iPad P..."
8,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,Google Pixel 8A Impressions: Just Get The 8!,B3szaVzQx0o,2024-05-07T16:18:33Z,PT7M23S,2433168,83348,4546,"['Pixel 8A', 'Pixel 8a', 'Pixel 8A impressions..."
9,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,Rabbit R1: Barely Reviewable,ddTV12hErTc,2024-04-30T00:21:20Z,PT19M53S,7769118,232705,21863,"['Rabbit R1', 'Rabbit vs', 'Rabbit R1 vs', 'Hu..."


In [121]:
comment_df

,Channel Name,Channel ID,Video Title,Video ID,Author Name,Authoer ID,Parent Comment ID,Comment ID,Text,Date Published,Date Updated,No. Likes
0,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,Talking Tech and AI with Tim Cook!,pMX2cQdPubk,@JOELLZE,UCgFuet1BNypqzyKt_HppOdg,UgzEswXILrJDcnyEhD94AaABAg,UgzEswXILrJDcnyEhD94AaABAg,Mentioning ‘Ergonomics’ and ‘Magic Mouse’ in t...,2024-06-12T16:01:58Z,2024-06-12T16:01:58Z,28728
1,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,Talking Tech and AI with Tim Cook!,pMX2cQdPubk,@awdadwadwad1723,UC-k2Bv11O8vSB2cp2Z0vPgg,UgzEswXILrJDcnyEhD94AaABAg,UgzEswXILrJDcnyEhD94AaABAg.A4aI05D0prbA4aIsDMpkWd,I literally loled at that moment,2024-06-12T16:09:29Z,2024-06-12T16:09:29Z,1143
2,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,Talking Tech and AI with Tim Cook!,pMX2cQdPubk,@daswasesheutegibt,UCMGmT09jCXicTVwjAM5YdXw,UgzEswXILrJDcnyEhD94AaABAg,UgzEswXILrJDcnyEhD94AaABAg.A4aI05D0prbA4aIyVyBXNd,He can’t be serious?,2024-06-12T16:10:21Z,2024-06-12T16:10:21Z,635
3,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,Talking Tech and AI with Tim Cook!,pMX2cQdPubk,@DomTheBom-ru6ib,UCYpvRAMhU2TRe2YEkbTozhA,UgzEswXILrJDcnyEhD94AaABAg,UgzEswXILrJDcnyEhD94AaABAg.A4aI05D0prbA4aJTmLjhHx,Was thinking the same thing lmao,2024-06-12T16:14:45Z,2024-06-12T16:14:45Z,214
4,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,Talking Tech and AI with Tim Cook!,pMX2cQdPubk,@Zagoorland,UCXn-BSSWoCmdbTYT8QlluVg,UgzEswXILrJDcnyEhD94AaABAg,UgzEswXILrJDcnyEhD94AaABAg.A4aI05D0prbA4aJU0gLtlc,Right?? 😅😅😅,2024-06-12T16:14:47Z,2024-06-12T16:14:47Z,82
...,...,...,...,...,...,...,...,...,...,...,...,...
32115,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,Reviewing EVERY Samsung Galaxy S Ever!,eKVTFXQPAhs,@rugrust08,UCmrdz9oRBu2OaV8K04_Oacw,Ugy8vXl8WRIC3I1__Z54AaABAg,Ugy8vXl8WRIC3I1__Z54AaABAg,Samsung galaxy s9 is poop 💩 phone ever I am us...,2024-06-04T08:06:47Z,2024-06-04T08:06:47Z,1
32116,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,Reviewing EVERY Samsung Galaxy S Ever!,eKVTFXQPAhs,@tatsugiri1,UC-7M5My6wsVl488zYk_L-8Q,Ugy5PiuR3gNzGF9bT-B4AaABAg,Ugy5PiuR3gNzGF9bT-B4AaABAg,s20 is the ugliest galaxy ever 😭😭,2024-03-31T11:53:14Z,2024-03-31T11:53:14Z,1
32117,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,Reviewing EVERY Samsung Galaxy S Ever!,eKVTFXQPAhs,@chrisjohn3992,UCNa2vie_znRCiiQ9if6zrIg,Ugy-oJIJHhRmSLNU9Qx4AaABAg,Ugy-oJIJHhRmSLNU9Qx4AaABAg,I own the worse Samsung Galaxy ever..s20fe,2024-03-30T10:58:38Z,2024-03-30T10:58:38Z,0
32118,Marques Brownlee,UCBJycsmduvYEL83R_U4JriQ,Reviewing EVERY Samsung Galaxy S Ever!,eKVTFXQPAhs,@Daniel-hy7th,UCY12zmqaofI2PxyCS1wXi1Q,UgxLc6Cys37kt77CuZ54AaABAg,UgxLc6Cys37kt77CuZ54AaABAg,All S20FE users gather here >>>>>>>>>>>>>,2024-03-21T12:40:25Z,2024-03-21T12:40:25Z,0


### 2. Data Cleaning & Preprocessing

### 3. TF-IDF

### 4.

### 5.

### 6.